In [1]:
#1
import numpy as np
%pylab 
#style.use('ggplot')
#rcParams['figure.figsize'] = 12,8
import numba as nb
from numba import jit

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
import copy
import networkx as nx
from copy import copy, deepcopy



import pandas as pd

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
#import connectivity as ct

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


In [2]:
pip install catboost

Note: you may need to restart the kernel to use updated packages.


In [3]:
from catboost import CatBoostClassifier, Pool, cv

Matrix connectivity functions

In [4]:
#Erdos-Renyi connectivity matrix 
@jit(nopython = True)
def getRandomConnectivity(N, pct_connected):
    #Directed mapping
    M = np.random.rand(N**2).reshape(N,N)
    for i in range(N):
        for j in range(N):
            if (i == j):
                M[i,j] = 0
            else:
                if (M[i,j] < pct_connected):
                    M[i,j] = 1 #i connects to j
                else:
                    M[i,j] = 0
    return M

@jit(nopython = True)
def kill_reciprocal(N,M):
  for i in range(N):
    for j in range(i):
      if ((M[i,j] == 1) and (M[j,i] == 1)):
        q =np.random.randint(2)
        if (q==0):
          M[i,j] = 0
        else:
          M[j,i] = 0
  return(M)






#Addition of motives by completing the triangle
@jit(nopython=True)
def addmotives(N,M,l,nummotives):
    for m in range(nummotives):
        c = 0
        while (c == 0):
            a = 0
            while(a < 2):
                i = np.random.randint(N)
                a = int(l[i,0])
            j1 = 0
            j2 = 0
            while (j1 == j2):
                j1 = np.random.randint(a)+1
                j2 = np.random.randint(a)+1
            b = int(l[j1,0])
            d = 0
            for k in range(b):
                if (j2 == int(l[j1,k+1])):
                    d = 1
            if (d==0):
                l[j1,0] = l[j1,0] + 1
                l[j1,b + 1] = j2
                c=1
    return(l)
        


#Addition of motives by adding full triangle    
def addmotives2(N,M,nummotives):
    m = 0
    while(m<nummotives):
        i = np.random.randint(N)
        j = np.random.randint(N)
        k = np.random.randint(N)
        if (M[i,j]==0):
            M[i,j] = 1
            m = m + 1
        if (M[j,k]==0):
            M[j,k] = M[j,k] + 1
            m = m + 1
        if (M[k,i] == 0):
            M[k,i] = M[k,i]+1
            m = m + 1
    #l = matrixOfEdges(M,N)
    return(M)        

def numberofmotives(N,M,l):
    num = 0
    for i in range(N):
        a = int(l[i,0])
        for j in range(a):
            for k in range(a):
                l1 = int(l[i,j+1])
                l2 = int(l[i,k+1])
                if (M[l1,l2]==1):
                    num = num + 1
    return(num)





def findandkillmotives(N,M,l,numtokill):
    num = 0
    i = 0
    while ((i<N)and(num < numtokill)):
        a = int(l[i,0])
        for j in range(a):
            for k in range(a):
                l1 = int(l[i,j+1])
                l2 = int(l[i,k+1])
                if (M[l1,l2]==1):
                    num = num + 1
                    M[l1,l2] = 0
        i = i + 1
    return(M,num)
    
def getrandomvector(N,ics):
    V=zeros(N)
    p=ics*1.0/N
    for i in range(N):
        if (np.random.rand(1)<p):
            V[i]=1
    return(V)



@jit(nopython=True)
def matrixOfEdges(M,N):
    N = int(N)
    E=np.zeros(N**2).reshape(N,N)  #prepare array with zeros
    k=np.sum(M,0) #array with degree of each vertex
    for i in range(N):
        a=int(k[i]) #degree of the current vertex
        E[i,0]=a  #we put it to the zero row of matrix of edges
    for i in range(N):
        a=int(k[i])
        q=1
        for j in range(N):
            if (M[j,i]==1):
                E[i,q]=j #all the next elements in current column are number of vertices current vertex is connected to
                q=q+1
    return(E)


@jit(nopython=True)
def recovermatrixfromlist(E,N):
    q=0;
    M=np.zeros(N*N).reshape(N,N)
    for i in range(N):
        a=E[i,0]
        b=int(a)
        for j in range(b):
            k=E[i,j+1]
            l=int(k)
            M[i,l]=1
    return(M)


def doubleconstellation(N,k):
    M=np.zeros(N*N).reshape(N,N)
    for i in range(k):
        for j in range(k):
            M[i,j+k]=1            
    for i in range(N-2*k):
        for j in range(k):
            M[i+2*k,j]=1
            M[j+k,i+2*k]=1
    return(M)



def constellationwithrandom(N,k,p):
    Edges=(2*(N-2*k)*k-k*k)*1.0
    M=np.zeros(N*N).reshape(N,N)
    for i in range(k):
        for j in range(k):
            M[i,j+k]=1            
    for i in range(N-2*k):
        for j in range(k):
            M[i+2*k,j]=1
            M[j+k,i+2*k]=1
    p=p-Edges*1.0/(N*N-N)
    Q = np.random.rand(N**2).reshape(N,N)
    for i in range(N):
        for j in range(N):
            if (i == j):
                M[i,j] = 0
            else:
                if (Q[i,j] < p):
                    M[i,j] = 1 #i connects to j
    return(M)


def constantdegree(N,k):
    M=zeros(N*N).reshape(N,N)
    for i in range(N):
        for j in range(k):
            m=np.random.randint(N)
            M[i,m]=1
    return(M)


def coordinates(A,m,d):
    X=A
    coords=np.zeros(d)
    remainder=np.zeros(d)
    for i in range(d):
        (X,coords[i])=divmod(X,m)
    return(coords)

def distance(A,B,m,d):
    coordsa = coordinates(A,m,d)
    #print(coordsa)
    coordsb = coordinates(B,m,d)
    #print(coordsb)
    vecdist=np.zeros(d)
    for i in range(d):
        vecdist[i]=(coordsa[i]-coordsb[i])**2
    dist=np.sum(vecdist)
    return(dist)

def localmatr(m,d,p,a):
    N=m**d
    M = np.random.rand(N**2).reshape(N,N)
    for i in range(N):
        for j in range(N):
            if (i == j):
                M[i,j] = 0
            else:
                if (M[i,j] < p*np.exp(-a*distance(i,j,m,d))):
                    #print(distance(i,j,m,d))
                    M[i,j] = 1 #i connects to j
                else:
                    M[i,j] = 0
    return M

    


def ranvec(N,n):
    vec=np.zeros(N)
    for i in range(n):
        a=np.random.randint(N)
        while(vec[a]==1):
            a=np.random.randint(N)
        vec[a]=1
    return(vec)

def random_initial_state(N,n_times,n):
    states = np.zeros(N*n_times).reshape(N,n_times) - 1
    vec = ranvec(N,n)
    for i in range(N):
        states[i,0] = -1  - vec[i]
    return(states)

def add_one_active(N,vec):
    i = np.random.randint(N)
    while(vec[i] == 1):
        i = np.random.randint(N)
    vec[i] = 1
    return(vec)

def vector_to_matrix(N,n_times,vec):
    states = np.zeros(N*n_times).reshape(N,n_times) - 1
    for i in range(N):
        states[i,0] = -1 - vec[i]
    return(states)


def spike_raster_plot(N, n_timesteps, stepsize, states):
    neuralData = []
    for i in range(N):
        neuralData.append([])
        for j in range(n_timesteps):
            if (states[i,j] == 0):
                neuralData[i].append(j*stepsize)
    return(neuralData)


Dynamic functions, the simplest approach: background noise just contributes to the potential. We assume it to contribute equally to all neurons, we do not take here into account variations in the noise, we assume all the neurons to be identical.The growth of the EPSP is instanateous in this model.   

In [5]:
#Here we are trying another approach by taking into account synaptic delay.

#The function that updates neuron's state based on its current state and input.

#We are not modeling laser activated neurons as separate entities 
#but just as the same neuronsvwith additional input
@jit(nopython=True)
def neuron_state_update(state, inp, voltage, refr, threshold, EPSP_decay, tau_m):
    V = voltage * EPSP_decay + inp / tau_m
    if ((state < refr) or (V < threshold)):
        return(state + 1, V)
    else: 
        return(0,0)


@jit(nopython=True)
def neuron_state_update_noise(state, inp, voltage, refr, threshold, EPSP_decay, tau_m, noise_freq):
    V = voltage * EPSP_decay + inp / tau_m
    to_fire = np.random.rand()
    if (((state < refr) or (V < threshold)) and (to_fire > noise_freq) ):
        return(state + 1, V)
    else: 
        return(0,0)


@jit(nopython=True)
def neuron_state_update_2(state, inp, voltage, refr, threshold, EPSP_decay, tau_m):
    V = voltage * EPSP_decay + inp / tau_m
    if ((state < refr) or (V < threshold)):
        return(state + 1, V,0)
    else: 
        return(0,0,0)



@jit(nopython=True)
def neuron_state_update_noise_2(state, inp, voltage, refr, threshold, EPSP_decay, tau_m, noise_freq):
    V = voltage * EPSP_decay + inp / tau_m
    to_fire = np.random.rand()
    if (((state < refr) or (V < threshold)) and (to_fire > noise_freq) ):
        return(state + 1, V,0)
    elif (to_fire <= threshold): 
        return(0,0,1)
    
@jit(nopython=True)    
def neuron_output(state,n_neighbors, W_synapse, tau_s, mu,  sigma):
    for i in range(n_neighbors):
        if (state == 0):
            W_synapse[i] = np.random.lognormal(mu, sigma)
            print(W_synapse[i])
    return(W_synapse*state*np.exp(-state / tau_s), W_synapse)
    
    

#It is not real excitation, but effective one
@jit(nopython=True) 
def excitation_by_laser(V, mu, sigma, n_spikes, n_timesteps):
    output = np.zeros(n_timesteps)
    j = 0
    for i in range(n_spikes):
        per = int(np.random.lognormal(mu, sigma))
        j = j + per
        output[j] = V
    return(output)


@jit(nopython=True) 
def excitation_by_laser_2(V, period, deviation, stepsize, n_spikes, n_timesteps):
    output = np.zeros(n_timesteps)
    j = int(np.random.normal(20 / stepsize, 3 / stepsize))
    output[j] = V
    for i in range(n_spikes - 1):
        per = int(np.random.normal(period / stepsize, deviation / stepsize))
        j = j + per
        output[j] = V
    return(output)

@jit(nopython = True)
def one_step_LIF(state, input_matrix, voltage,syn_del, W_syn,  edges, t, parameters):
    EPSP_decay = parameters[0] # we multiply voltage to this constant every step to get exponential decay
    refr = int(parameters[1]) 
    mu = parameters[2] 
    sigma = parameters[3]
    tau_s = int(parameters[4])
    tau_m = int(parameters[5])
    threshold = parameters[7] #threshold for the transition
    n_neurons = int(parameters[8]) #total number of neurons
    
    for i in range(n_neurons):
        statei = state[i,t]
        n_neighbors = int(edges[i,0])
        W_synapse = np.zeros(n_neighbors)
        out = np.zeros(n_neighbors)
        constant = statei*np.exp(-statei / tau_s)
        for j in range(n_neighbors):
            out[j] = W_syn[i,j]*constant
        #out, W_synapse = neuron_output(statei,n_neighbors, W_synapse, tau_s, mu,  sigma)
            neighbor = int(edges[i, j+1])
            delay = int(syn_del[i,j])
            input_matrix[neighbor, t + delay + 1] = input_matrix[neighbor, t + delay + 1] + out[j]
    for i in range(n_neurons):
        state[i, t + 1], voltage[i, t + 1] = neuron_state_update(state[i, t ], input_matrix[i, t], voltage[i, t ], refr, threshold, EPSP_decay,tau_m)
        
    return(state, input_matrix, voltage)

@jit(nopython = True)
def create_noise_vector(N, fraction):
  nv = np.zeros(N)
  for i in range(N):
    p = np.random.rand()
    if (p < fraction):
      nv[i] = 1
  return(nv)

@jit(nopython = True)
def one_step_LIF_noise(state, input_matrix, voltage,syn_del, W_syn,  edges, t, parameters, noise_freq, noise_vector, activated_by_noise):
    EPSP_decay = parameters[0] # we multiply voltage to this constant every step to get exponential decay
    refr = int(parameters[1]) 
    mu = parameters[2] 
    sigma = parameters[3]
    tau_s = int(parameters[4])
    tau_m = int(parameters[5])
    threshold = parameters[7] #threshold for the transition
    n_neurons = int(parameters[8]) #total number of neurons
  
    
    for i in range(n_neurons):
        statei = state[i,t]
        n_neighbors = int(edges[i,0])
        W_synapse = np.zeros(n_neighbors)
        out = np.zeros(n_neighbors)
        constant = statei*np.exp(-statei / tau_s)
        for j in range(n_neighbors):
            out[j] = W_syn[i,j]*constant
        #out, W_synapse = neuron_output(statei,n_neighbors, W_synapse, tau_s, mu,  sigma)
            neighbor = int(edges[i, j+1])
            delay = int(syn_del[i,j])
            input_matrix[neighbor, t + delay + 1] = input_matrix[neighbor, t + delay + 1] + out[j]
    for i in range(n_neurons):
      if (noise_vector[i] == 1):
        state[i, t + 1], voltage[i, t + 1], activated_by_noise[i, t+1] = neuron_state_update_noise_2(state[i, t ], input_matrix[i, t], voltage[i, t ], refr, threshold, EPSP_decay, tau_m, noise_freq)
      else:
        state[i, t + 1], voltage[i, t + 1], activated_by_noise[i, t+1] = neuron_state_update_2(state[i, t ], input_matrix[i, t], voltage[i, t ], refr, threshold, EPSP_decay, tau_m)
  

    return(state, input_matrix, voltage, activated_by_noise)

@jit(nopython=True)
def prepare_syndel(average, std, stepsize, edges, n_neurons ):
    mu, sigma = prepare_lognormal(average, std, stepsize)
    synaptic_del = np.zeros(n_neurons**2).reshape(n_neurons, n_neurons)
    for i in range(n_neurons):
        for j in range(n_neurons - 1):
            if (edges[i, j + 1] > 0):
                 syndel = np.random.lognormal(mu, sigma)
    return(synaptic_del)


@jit(nopython=True)
def prepare_syndel_2(average, std, stepsize, edges, n_neurons ):
    synaptic_del = np.zeros(n_neurons**2).reshape(n_neurons, n_neurons)
    for i in range(n_neurons):
      neigh = int(edges[i,0])
      for j in range(neigh):
        synaptic_del[i,j] = np.random.uniform(average - std, average + std)
    return(synaptic_del)


@jit(nopython = True)
def prepare_input(V, period, std, stepsize, n_neurons, n_timesteps, n_spikes, set_activated):
    input_matrix = np.zeros(n_neurons*n_timesteps).reshape(n_neurons, n_timesteps)
    n_activated = len(set_activated)
    mu, sigma = prepare_lognormal(period, std, stepsize)
    for i in range(n_activated):
        vec = excitation_by_laser(V, mu, sigma, n_spikes, n_timesteps)
        number = int(set_activated[i])
        for j in range(n_timesteps):
            input_matrix[number, j] = vec[j]
    return(input_matrix)
        
    
@jit(nopython = True)
def prepare_input2(V, period, std, stepsize, n_neurons, n_timesteps, n_spikes, set_activated):
    input_matrix = np.zeros(n_neurons*n_timesteps).reshape(n_neurons, n_timesteps)
    n_activated = len(set_activated)
    for i in range(n_activated):
        vec = excitation_by_laser_2(V, period, std, stepsize, n_spikes, n_timesteps)
        number = int(set_activated[i])
        for j in range(n_timesteps):
            input_matrix[number, j] = vec[j]
    return(input_matrix)
        
@jit(nopython = True)
def compact_input(input_matrix, set_activated, n_timesteps, n_spikes):
    
    n_activated = len(set_activated)
    
    inp = np.zeros(n_activated * n_spikes).reshape(n_activated, n_spikes)
    for i in range(n_activated):
        number = int(set_activated[i])
        num_of_spike = 0
        for j in range(n_timesteps):
            
            #print(type(num_of_spike))
            if (input_matrix[number, j] > 0):
                k = int(num_of_spike)
                inp[i, k] = j
                num_of_spike =num_of_spike + 1
             #   print(type(num_of_spike))
    #print(inp)
    
    return(inp)
        
    

                
#The full process of synchronization, starting with particular set of activated neurons           
@jit(nopython=True)
def full_process( edges, syn_del, input_matrix, W_matrix,  parameters, n_timesteps, noise_freq, noise_vector):
    n_neurons = int(parameters[8])
    stepsize = parameters[9]
    
    tau_s = int(parameters[4])
    activated_by_noise = np.zeros(n_neurons*n_timesteps).reshape(n_neurons, n_timesteps)
    states = np.zeros(n_neurons*n_timesteps).reshape(n_neurons, n_timesteps)
    for i in range(n_neurons):
        for j in range(n_timesteps):
            states[i,j] = 1000 * tau_s
    
                
    #        print(W_matrix[i,j])
    #excitation_by_laser(V, period, std, n_spikes, n_timesteps)
    max_del = np.max(syn_del.reshape(n_neurons**2))
    V_threshold = parameters[7] 
    voltages = np.zeros(n_neurons*n_timesteps).reshape(n_neurons, n_timesteps)
    t = 0
    av_V = 0
    while ((t < n_timesteps - 1 - max_del) and (av_V < V_threshold)):
        states, input_matrix, voltages, activated_by_noise = one_step_LIF_noise(states, input_matrix, voltages, syn_del, W_matrix,  edges, t, parameters, noise_freq, noise_vector, activated_by_noise)
        t = t + 1
        sum_V = 0
        for i in range(n_neurons):
            sum_V = sum_V + voltages[i,t]
        av_V = sum_V / n_neurons    
        #print(av_V)
    return(states, voltages, activated_by_noise, t)    



@jit(nopython=True)
def full_process_data( edges, syn_del,  W_matrix,  parameters, n_timesteps, noise_freq, noise_vector, V_laser, period, stdlaser, n_spikes, n_activated):
    n_neurons = int(parameters[8])
    stepsize = parameters[9]
    
    tau_s = int(parameters[4])
    activated_by_noise = np.zeros(n_neurons*n_timesteps).reshape(n_neurons, n_timesteps)
    states = np.zeros(n_neurons*n_timesteps).reshape(n_neurons, n_timesteps)
    for i in range(n_neurons):
        for j in range(n_timesteps):
            states[i,j] = 1000 * tau_s
    
    
    set_activated = active_set(N, n_activated) #activated neurons are chosen
                
    #        print(W_matrix[i,j])
    #excitation_by_laser(V, period, std, n_spikes, n_timesteps)
    max_del = np.max(syn_del.reshape(n_neurons**2))
    V_threshold = parameters[7] 
    voltages = np.zeros(n_neurons*n_timesteps).reshape(n_neurons, n_timesteps)
    t = 0
    av_V = 0
    input_matrix = prepare_input2(V_laser, period, stdlaser, stepsize, n_neurons, n_timesteps, n_spikes, set_activated)
    inp = compact_input(input_matrix, set_activated, n_timesteps, n_spikes)
    while ((t < n_timesteps - 1 - max_del) and (av_V < V_threshold)):
        states, input_matrix, voltages, activated_by_noise = one_step_LIF_noise(states, input_matrix, voltages, syn_del, W_matrix,  edges, t, parameters, noise_freq, noise_vector, activated_by_noise)
        t = t + 1
        sum_V = 0
        for i in range(n_neurons):
            sum_V = sum_V + voltages[i,t]
        av_V = sum_V / n_neurons    
        #print(av_V)
    if (t < n_timesteps - 1 - max_del):
        burst = 1
    else:
        burst = 0
    
    return(inp, burst)  



        
@jit(nopython=True)           
def prepare_lognormal(av, std,stepsize):   
    sigmasq = np.log((std/av)**2 + 1) 
    sigma = np.sqrt(sigmasq)
    mu = np.log(av / stepsize)  - sigmasq / 2.0
    return(mu,sigma)    
    

@jit(nopython=True)                
def turn_data_into_scatter(N, n_tim, state):
    x = []
    y = []
    for j in range(n_tim):
        for i in range(N):
            if (state[i,j] == 0):
                x.append(i)
                y.append(j)
    return(x,y)
    
                
def probs_times(res, n_timesteps, n_trials, n_curves, n_max, stepsize):
    times = []
    probs = []
    for c in range(n_curves):
        t = res[c]
        times.append([])
        probs.append([])
        n_act = []
        for i in range(n_max):
            n_burst = 0
            sum_t = 0
            for j in range(n_trials):
                if (t[i,j] < n_timesteps - 1):
                    n_burst = n_burst + 1
                    sum_t = sum_t + t[i,j]                
            probs[c].append(n_burst * 1.0 / n_trials)
            if (n_burst > 0):
                sum_t = sum_t / n_burst
                times[c].append(sum_t * stepsize)
    return(times, probs)                     
            
def plotter(times, probs, n_curves, n_max):
    n_act = []
    n_shifted = np.zeros(n_max)
    for i in range (n_max):
        n_shifted[i] = i + 1
    for i in range(n_curves):
        
        q = len(times[i])
        n_act_vec = np.zeros(q)
        for j in range(q):
            n_act_vec[j] = n_max - q + j + 1
        n_act.append(n_act_vec)
    figure(1)
    for i in range(n_curves):
        plot(n_shifted, probs[i])
    figure(2)
    for i in range(n_curves):
        plot(n_act[i],times[i])
    
    
    
    
def list_from_matrices(N, edges, syn, W):
  e_l = []
  s_l = []
  W_l = []
  for i in range(N):
    neigh = int(edges[i,0])
    e_l.append([])
    s_l.append([])
    W_l.append([])
    for j in range(neigh):
      e_l[i].append(edges[i,j+1])
      s_l[i].append(syn[i,j])
      W_l[i].append(W[i,j])
  return(e_l,s_l,W_l)


def weight_matrix(n_neurons, edges, mu,sigma): 
   W_matrix = np.zeros(n_neurons**2).reshape(n_neurons, n_neurons)
   for i in range(n_neurons):
     neigh = int(edges[i,0])
     for j in range(neigh):
      W_matrix[i,j] = np.random.lognormal(mu, sigma)
   return(W_matrix)

@jit(nopython = True)
def active_set(N, n_activated):
  set_activated = np.zeros(n_activated)
  for i in range(n_activated):
    a = np.random.randint(N)
    for j in range(i):
      while (a == set_activated[j]):
        a = np.random.randint(N)
    set_activated[i] = a
  return(set_activated)    


def catboost_data_full_model(data_size, edges, syn_del, W_matrix,  parameters, n_timesteps, noise_freq, noise_vector, V_laser, period, stdlaser, n_spikes, n_activated):
   
    my_dict = {}
    inp, burst = full_process_data( edges, syn_del, W_matrix,  parameters, n_timesteps, noise_freq, noise_vector, V_laser, period, stdlaser, n_spikes, n_activated)
    for k in range(len(inp)):
        my_dict[k] = []
    my_dict["Answer"] = []
    
    #print(my_dict)
    #X = np.zeros(data_size * num_par).reshape(data_size, num_par) #for not rescaling
    #X = []
    #Y = []
    #perm_size = math.factorial(N)
    #pl1 = list(np.arange(N))
    #pl_all = list(permutations(pl1)) 
    for i in range(data_size):
        
        inp, burst = full_process_data( edges, syn_del, W_matrix,  parameters, n_timesteps, noise_freq, noise_vector, V_laser, period, stdlaser, n_spikes, n_activated)

        
        for k in range(len(inp)):
            my_dict[k].append(inp[k]) 

        my_dict["Answer"].append(burst)
            
    
    
    
    return(my_dict)  


@jit(nopython = True)
def single_EPSP_magnitude(EPSP_decay, tau_s ,n_steps):
    V = np.zeros(n_steps)
    for i in range(1,n_steps):
        V[i] = V[i - 1] * EPSP_decay + i * exp(-i*1.0/tau_s) / tau_m 
    return(np.max(V))    


@jit(nopython = True)
def multiple_EPSP_magnitude(delV, tau_m, deltaT):
    return(delV/ ( 1 - np.exp(- deltaT / tau_m )))


@jit(nopython = True)
def minimal_interval(inp, n_spikes, n_activated):
    deriv = np.zeros(n_activated * (n_spikes - 1)).reshape(n_activated, n_spikes - 1)
    for i in range(n_activated):
        for j in range(n_spikes - 1):
            deriv[i,j] = inp[i, j + 1] - inp[i,j]
    A = deriv.reshape(n_activated * (n_spikes - 1))
    return(np.min(A))


        
@jit(nopython=True)
def lists_to_matrices(N, edges, W, syn_del):
    
    W_mat = np.zeros(N*N).reshape(N,N)
    syn_mat = np.zeros(N*N).reshape(N,N)

    for i in range(N):
        n_neighbors = int(edges[i,0])
        for j in range(n_neighbors):
            q = int(edges[i, j + 1])
            W_mat[i,q] = W[i, j]
            syn_mat[i,q] = syn_del[i,j]
    return(W_mat,syn_mat)    
        
        
@jit(nopython=True)
def lists_to_matrices_2(N, edges, W, syn_del):
    
    W_mat = np.zeros(N*N).reshape(N,N)
    syn_mat = np.zeros(N*N).reshape(N,N)

    for i in range(N):
        n_neighbors = int(edges[i,0])
        for j in range(n_neighbors):
            q = int(edges[i, j + 1])
            W_mat[q,i] = W[i, j]
            syn_mat[q,i] = syn_del[i,j]
    return(W_mat,syn_mat)    
            
    

@jit(nopython = True)
def maybe_active(M, edges, syn_del, inp, set_activated, W_matrix,  parameters, n_timesteps, noise_freq, noise_vector, n_spikes):
    N = int(parameters[8])
    n_activated = len(set_activated)
    V_threshold = parameters[7]
    EPSP_decay = parameters[0]  

    tau_s = parameters[4] 
    tau_m = parameters[5]
    stepsize = parameters[9]
    n_steps = int(5.0 / stepsize)
    delV = single_EPSP_magnitude(EPSP_decay, tau_s , n_steps)
    #print(delV)
    
    deltaT = minimal_interval(inp, n_spikes, n_activated)
    #print(deltaT)
    max_possible_inp = np.zeros(N)
    num_inp = np.zeros(N)
    for i in range(n_activated):
        n_i = int(set_activated[i])
        n_neigh = edges[n_i, 0]
        for j in range(n_neigh):
            k = int(edges[n_i, j+1])
            max_possible_inp[k] += W_matrix[n_i,j] * multiple_EPSP_magnitude(delV, tau_m, deltaT)
            num_inp[k] += 1
    num_potentially_active = 0
    #print(max_possible_inp)
    for i in range(N):
        if (max_possible_inp[i] > V_threshold):
            num_potentially_active += 1
    potentially_active = np.zeros(num_potentially_active)
    num_inp_above = np.zeros(num_potentially_active)
    inp_values = np.zeros(num_potentially_active)
    index = 0
    for i in range(N):
         if (max_possible_inp[i] > V_threshold):
                potentially_active[index] = i
                num_inp_above[index] = num_inp[i]
                inp_values[index] = max_possible_inp[i]
                index += 1
    return(potentially_active, num_inp_above, inp_values)
        
                

@jit(nopython = True)
def small_matrices(pot_act, Matr_itself, set_activated, edges, W_mat, syn_mat):
    n_active = len(set_activated)
    n_pot = len(pot_act)
    M_small = np.zeros(n_pot * n_active).reshape(n_pot, n_active)
    Edge_small = np.zeros(n_pot * n_active).reshape(n_pot, n_active)
    W_small = np.zeros(n_pot * n_active).reshape(n_pot, n_active)
    W_mat_small = np.zeros(n_pot * n_active).reshape(n_pot, n_active)
    syn_small = np.zeros(n_pot * n_active).reshape(n_pot, n_active)
    syn_mat_small = np.zeros(n_pot * n_active).reshape(n_pot, n_active)
    for i in range(n_activated):
        for j in range(n_pot):
            a = int(set_activated[i])
            b = int(pot_act[j])
            if (Matr_itself[b,a] == 1):
                M_small[j,i] = 1
                W_mat_small[j,i] = W_mat[a,b]
                syn_mat_small[j,i] = syn_mat[a,b]
    deg=np.sum(M_small,1) #array with degree of each vertex
    deg2 = np.sum(M_small,0)
    #print(deg)
    #print(deg2)
    for i in range(n_pot):
        a=int(deg[i]) #degree of the current vertex
        Edge_small[i,0]=a  #we put it to the zero row of matrix of edges
    for i in range(n_pot):
        #a=int(deg[i])
        q=1
        for j in range(n_active):
            if (M_small[i,j]==1):
                Edge_small[i,q]=j #all the next elements in current column are number of vertices current vertex is connected to
                W_small[i,q-1] = W_mat_small[i,j]
                syn_small[i,q-1] = syn_mat_small[i,j]
                q=q+1
    
    return(M_small, Edge_small, W_small, syn_small)
                

def list_sort_together(list1, list2):
    list1, list2 = (list(t) for t in zip(*sorted(zip(list1, list2))))
    return(list1, list2)
    
#@jit(nopython = True)
def really_active(M_small, Edge_small, W_small, syn_small, inp,  parameters, n_spikes, n_activated, n_pot):
    Spikes_list = [] #= np.zeros(n_pot * n_activated * n_spikes).reshape(n_pot, n_activated * n_spikes)
    V_threshold = parameters[7]
    EPSP_decay = parameters[0]  
    stepsize = parameters[9]
    tau_s = parameters[4] 
    tau_m = parameters[5]
    n_steps = int(5.0 / stepsize)
    delV = single_EPSP_magnitude(EPSP_decay, tau_s ,n_steps)
    for i in range(n_pot):
        inp_volts = []
        volts = [0]
        
        times = [0]
        
        index = []
        spikes = []
        k = int(Edge_small[i,0])
        for j in range(n_spikes):
            Inp_volts = []
            Times = []
            for q in range(k):
                out_neur = int(Edge_small[i, q + 1])
                out_time = inp[out_neur, j]
                Times.append(out_time + syn_small[i, q ]) 
                #print("inp-append", W_small[i, q ])
                Inp_volts.append(W_small[i, q ] * delV)
                index.append(j)
                volts.append(0)
            Times, Inp_volts = list_sort_together(Times, Inp_volts)
            inp_volts += Inp_volts
            times += Times
        #print(inp_volts)
        #print(times)
        for j in range(1, len(times)):
         #   print("Inp = ", inp_volts[j - 1])
            volts[j] = volts[j-1] * np.exp(-(times[j] - times[j-1]) / tau_m) + inp_volts[j - 1]
            if (volts[j] > V_threshold):
                spikes.append(index[j - 1])
                volts[j] = 0
          #  print("Volts = ",volts[j])
        #print(volts)
        #print(spikes)
                
        Spikes_list += spikes
        
    counter = np.zeros(n_spikes)
    for i in range(len(Spikes_list)):
        counter[Spikes_list[i]] += 1
        
        
        
        
    return(counter)


def temporal_data_same(data_size,edges, syn_del, W_matrix,  parameters, n_timesteps, noise_freq, noise_vector, V_laser, period, stdlaser, n_spikes, n_activated):
    my_dict = {}
    #for k in range(n_spikes):
    #    my_dict[k] = []
    my_dict["Input"] = []
    my_dict["Answer"] = []
    for i in range(data_size):
        delV = single_EPSP_magnitude(EPSP_decay, tau_s , 1000)
        input_matrix = prepare_input2(V_laser, period, stdlaser, stepsize, n_neurons, n_timesteps, n_spikes, set_activated)
    #print(3)
        inp = compact_input(input_matrix, set_activated, n_timesteps, n_spikes)
        W_mat, syn_mat = lists_to_matrices(N, edges, W_matrix, syn_del)
    #inp, b = full_process_data( edges, syn_del, W_matrix,  parameters, n_timesteps, noise_freq, noise_vector, V_laser, period, stdlaser, n_spikes, n_activated)
        A,B,C, t = full_process( edges, syn_del, input_matrix, W_matrix,  parameters, n_timesteps, noise_freq, noise_vector)
        if (t < n_timesteps - np.max(syn_del) - 1):
            b = 1
        else:
            b = 0
        pot_act, num_inp, inp_values = maybe_active(M, edges, syn_del, inp, set_activated, W_matrix,  parameters, n_timesteps, noise_freq, noise_vector, n_spikes)
        
        M_small, Edge_small, W_small, syn_small = small_matrices(pot_act, M, set_activated, edges, W_mat, syn_mat)
   
        n_activated = len(set_activated)
        n_pot = len(pot_act)
        res = really_active(M_small, Edge_small, W_small, syn_small, inp,  parameters, n_spikes, n_activated, n_pot)
        mres = np.max(res)
        #for k in range(n_spikes):
         #   my_dict[k].append(res[k])
            #my_dict[k].append(np.random.rand())
        my_dict["Input"].append(mres)    
        my_dict["Answer"].append(b)
        
    return(my_dict)
           
          
def temporal_data_dif_mat_dif_jit(data_size,  parameters, n_timesteps, noise_freq, noise_vector, V_laser, period, stdlaser, n_spikes, n_activated):
    my_dict = {}
    #for k in range(n_spikes):
    #    my_dict[k] = []
    my_dict["Weights"] = []
    #my_dict["Edges"] = []
    my_dict["std"] = []
    
    my_dict["Set activated"] = []
    
    my_dict["Answer"] = []
    
    my_dict["Times spiking"] = []
    for i in range(data_size):
        if (i%10 == 0):
            print(i)
        N = 1000 #total number of neurons

        n_neurons = N #the same thing
        parameters[8] = N

        p = 0.065 #probability of the directed connection


        M = getRandomConnectivity(N,p) #prepare connectivity matrix of the network. In this case ER matrix.
        edges = matrixOfEdges(M,N)  #other form of this matrix
        W_matrix = weight_matrix(n_neurons, edges, mu,sigma) #matrix of synaptic weights


        set_activated = active_set(N, n_activated)
        syn_del = prepare_syndel_2(avdel, stddel, stepsize, edges, n_neurons ) #matrix of synaptic delays
        #stdlaser = period * np.random.rand() / 3.0    
        delV = single_EPSP_magnitude(EPSP_decay, tau_s , 1000)
        input_matrix = prepare_input2(V_laser, period, stdlaser, stepsize, n_neurons, n_timesteps, n_spikes, set_activated)
    #print(3)
        inp = compact_input(input_matrix, set_activated, n_timesteps, n_spikes)
        W_mat, syn_mat = lists_to_matrices_2(N, edges, W_matrix, syn_del)
    #inp, b = full_process_data( edges, syn_del, W_matrix,  parameters, n_timesteps, noise_freq, noise_vector, V_laser, period, stdlaser, n_spikes, n_activated)
        A,B,C, t = full_process( edges, syn_del, input_matrix, W_matrix,  parameters, n_timesteps, noise_freq, noise_vector)
        if (t < n_timesteps - np.max(syn_del) - 1):
            b = 1
        else:
            b = 0
        pot_act, num_inp, inp_values = maybe_active(M, edges, syn_del, inp, set_activated, W_matrix,  parameters, n_timesteps, noise_freq, noise_vector, n_spikes)
        
        M_small, Edge_small, W_small, syn_small = small_matrices(pot_act, M, set_activated, edges, W_mat, syn_mat)
   
        n_activated = len(set_activated)
        n_pot = len(pot_act)
        #res = really_active(M_small, Edge_small, W_small, syn_small, inp,  parameters, n_spikes, n_activated, n_pot)
        #mres = np.max(res)
        #for k in range(n_spikes):
         #   my_dict[k].append(res[k])
            #my_dict[k].append(np.random.rand())
        #my_dict["Input"].append(mres) 
        my_dict["Weights"].append(W_mat)
        
        my_dict["std"].append(stdlaser)
    
        my_dict["Set activated"].append(set_activated)
    
        
    
        my_dict["Times spiking"].append(inp)
        
        my_dict["Answer"].append(b)
        
    return(my_dict)
           

    
    
    

    
@jit(nopython = True)
def order_n_vec(X,n,N):
    T = np.zeros(N)
    for i in range(N):
            T[i] = X[i]**n
    return(T)
            
def data_change(my_dict, N,n):
    my_dict[n] = []
    
    lW = my_dict["Weights"]
    lX = my_dict["Set activated"] 
    for i in range(len(lW)):
        X = np.zeros(N)
        n_act = len(lX[i])
        for j in range(n_act):
            a = int(lX[i][j])
            X[a] = 1
        X_new = np.matmul(lW[i], X)
        my_dict[n].append(np.sum(order_n_vec(X_new, n, N)))
    return(my_dict)

def data_change_2(my_dict, N,n):
    my_dict[n] = []
    
    lW = my_dict["Weights"]
    lX = my_dict["Set activated"] 
    for i in range(len(lW)):
        X = np.zeros(N)
        n_act = len(lX[i])
        for j in range(n_act):
            a = int(lX[i][j])
            X[a] = 1
        for j in range(n):
            X = np.matmul(lW[i], X)
        my_dict[n].append(np.sum(order_n_vec(X, 2, N)))
    return(my_dict)



def data_change_Alex(my_dict, N,n):
    for i in range(n):
        my_dict[i] = []
    
    lW = my_dict["Weights"]
    lX = my_dict["Set activated"] 
    for i in range(len(lW)):
        X = np.zeros(N*N).reshape(N,N)
        n_act = len(lX[i])
        for j in range(n_act):
            a = int(lX[i][j])
            X[a,a] = 1
        for j in range(n):
            my_dict[j].append(np.sum(X))
            X = np.matmul(lW[i], X)
    print(len(my_dict["Weights"]))
    for i in range(n):
        print(len(my_dict[i]))
    return(my_dict)

def data_change_Alex_inverse(my_dict, N,alpha):
    my_dict[alpha] = []
    
    
    lW = my_dict["Weights"]
    lX = my_dict["Set activated"] 
    for i in range(len(lW)):
        X = np.zeros(N*N).reshape(N,N)
        n_act = len(lX[i])
        for j in range(n_act):
            a = int(lX[i][j])
            X[a,a] = 1
        A =  np.matmul(lW[i], X)
        B = np.identity(N) - alpha * A 
        C = np.matmul(A, inv(B))
        my_dict[alpha].append(np.sum(C))
    
    return(my_dict)



def transform_data(my_dict, N,n):
    my_dict[n + 2000] = []
    l = len(my_dict[0])
    for i in range(l):
        X = np.zeros(N)
        for j in range(N):
            X[j] = my_dict[j][i]
        my_dict[n + 2000].append(np.sum(order_n_vec(X, n, N)))
    return(my_dict)


def all_possible_parameters(my_dict, N):
    first_powers = 4
    second_powers = 3
    idm = np.identity(N)
    for i in range(first_powers * 2 * second_powers):
        my_dict[i ] = []
    lW = my_dict["Weights"]
    lX = my_dict["Set activated"] 
    for i in range(len(lW)):
        X = np.zeros(N)
        n_act = len(lX[i])
        for j in range(n_act):
            a = int(lX[i][j])
            X[a] = 1
        V1 = np.matmul(lW[i],X)
        for s1 in range(first_powers):
            V2 = order_n_vec(V1, s1, N)
            for s2 in range(2):
                if (s2==0):
                    V3 = np.matmul(idm, V2)
                else:
                    V3 = np.matmul(lW[i], V2)
                for s3 in range(second_powers):
                    V4 = order_n_vec(V3, s3, N)
                    numb = s3 + s2*second_powers + s1 * 2 * second_powers
                    my_dict[numb] = np.sum(V4)
    return(my_dict)
                    
            
                    
                    
        
            
                

# Parameters of the simulation

In [6]:
stepsize = 0.05 #the size of the time step, in milliseconds
n_timesteps = 10000 #the total number of steps (if burst is reached simulation stops immediately after the burst)

# Parameters of the individual neurons

In [7]:
refr = 3 / stepsize # refractory period of the neuron. Number before division is in milliseconds 
tau_s = 0.5 / stepsize #tau_s from the equation for the current I = W * t * exp(-t/ tau_s). Number in ms.
tau_m = 25 / stepsize #tau_m membrane time constant in milliseconds
V_threshold = 12.4 #the threshold voltage in millivolts
EPSP_decay = np.exp(-1.0 / tau_m )  # the factor by which the voltage decays following the equations
V_EPSP = 2.8 #average magnitude of EPSP in mV
std_EPSP = 1.5 #standard deviation of EPSP in mV
W_average = 300 #approximate value of the parameter W in the equation for current (see above) in mV
sigmasq = np.log((std_EPSP/V_EPSP)**2 + 1) #square of parameter sigma in lognormal distribution 
sigma = np.sqrt(sigmasq) #parameter sigma in lognormal distribution
mu = np.log(W_average) - sigmasq / 2.0  #parameter mu in lognormal distribution
mu = mu + np.log(stepsize) #changing the stepsize goes into mu in this way
noise_freq = 0.0005 * stepsize # frequency over the noise in kHz (1/ms)
avdel = 1.3 / stepsize  #average synaptic delay time. In ms before division
stddel = 1.1 / stepsize #standard deviation of the synaptic delay time. In ms before division

# Creating the Erdos-Renyi network

In [8]:
N = 1000 #total number of neurons

n_neurons = N #the same thing

p = 0.065 #probability of the directed connection


M = getRandomConnectivity(N,p) #prepare connectivity matrix of the network. In this case ER matrix.
edges = matrixOfEdges(M,N)  #other form of this matrix
W_matrix = weight_matrix(n_neurons, edges, mu,sigma) #matrix of synaptic weights


    
syn_del = prepare_syndel_2(avdel, stddel, stepsize, edges, n_neurons ) #matrix of synaptic delays

e_l, s_l , W_l = list_from_matrices(N, edges, syn_del, W_matrix) #write all the matrices in a form convenient for output



#Creating localized network

# Parameters of the laser activation

In [9]:
 #laser activation is modeled as very strong  input to the neuron during one step requrring few times to make the neuron fire multiple times. 
N = 1000
V_laser = 20 * tau_m #magnitude of the input. Should be enough to make neuron to fire 
period = 39.2 #average period of spiking after laser stimulation in ms, taken from frequency in Kaiwen paper
stdlaser = 4.7 #standard deviation  of the period
n_spikes = 7 #number of spikes
n_activated = 7 #number of activated neurons
set_activated = active_set(N, n_activated) #activated neurons are chosen
#set activated is a list of numbers. You also can give it explicitly, for example 
#set_activated = [1,2,3,4,5,6,7] 



# Prepare the list with parameters

In [10]:
N = 1000
parameters = np.zeros(10)
parameters[0] = EPSP_decay 
parameters[1] = refr
parameters[2] = mu 
parameters[3] = sigma 
parameters[4] = tau_s 
parameters[5] = tau_m
parameters[7] = V_threshold 
parameters[8] = n_neurons
parameters[9] = stepsize

#Specifying which neurons can produce noise and which can not

In [11]:
noise_fraction = 0.0 #fraction of noicy neurons

noise_vector = create_noise_vector(N, noise_fraction)



In [12]:
#Customary libraries
import connectivities as cn #Different network connectivities
import physiological_neurons as ph #Dynamics of leaky integrate ad fire neurons
import parameters as pr #Parameters of the simulation 

def data_with_saving(data_size, first_powers, second_powers):
    
    par = pr.my_params()
    N = int(par['n_neurons']) #total number of neurons
    n_activated = int(par['Number of laser activated neurons'])
    n_timesteps = int(par['n_timesteps'])
    noise_fraction = 0.0 #fraction of noicy neurons

    noise_vector = ph.create_noise_vector(N, noise_fraction)

    my_dict = {}
    my_dict["Answer"] = []
    
    
    idm = np.identity(N)
    for i in range(first_powers * 2 * second_powers):
        my_dict[i] = []
        
    for i in range(data_size):
        
        M, edges, W_matrix, syn_del = cn.generate_network(par)
        
        
        

        set_activated = cn.active_set(N, n_activated) #activated neurons are chosen
    
        input_matrix = ph.prepare_input2(par, set_activated)
        
        X = np.zeros(N)
        
        if (i%10 == 0):
            print(i)
       

        
        W_mat, syn_mat = cn.lists_to_matrices_2(N, edges, W_matrix, syn_del)
        A,B, t = ph.full_process( edges, syn_del, input_matrix, W_matrix,  par, noise_vector)
        if (t < n_timesteps - np.max(syn_del) - 1):
            b = 1
        else:
            b = 0

        
        
        
        for j in range(n_activated):
            a = int(set_activated[j])
            X[a] = 1
        V1 = np.matmul(W_mat,X)
        for s1 in range(first_powers):
            V2 = order_n_vec(V1, s1 + 1, N)
            for s2 in range(2):
                if (s2==0):
                    V3 = np.matmul(idm, V2)
                else:
                    V3 = np.matmul(W_mat, V2)
                for s3 in range(second_powers):
                    V4 = order_n_vec(V3, s3 + 1, N)
                    numb = s3 + s2*second_powers + s1 * 2 * second_powers
                    totres = np.sum(V4)
                    my_dict[numb] = [totres]
        
        
        
        my_dict["Answer"] = b
        df = pd.DataFrame(my_dict)
        df.to_csv('data4.csv', mode='a')
        
    return(my_dict)
           

In [13]:
data_size = 100
first_powers = 4
second_powers = 3
data_multiple_matrix_N_1000_r_4 = data_with_saving(data_size, parameters, n_timesteps, noise_freq, noise_vector, V_laser, period, stdlaser, n_spikes, n_activated)  
print("Data obtained")



0
1
1
1
0
0
0
1
1
0
0
10
1
0
0
0
1
1
0
1
0
1
20
1
1
0
0
0
0
1
1
0
1
30
1
0
1
1
0
1
0
0
1
1
40
1
1
1
1
0
0
1
0
0
0
50
0
0
1
0
1
0
0
1
1
0
60
1
1
0
0
1
1
1
1
0
1
70
1
0
1
1
1
1
0
1
0
1
80
1
0
0
0
1
1
0
1
1
1
90
1
1
1
1
0
0
0
1
1
0
Data obtained


In [14]:

    
def multiple_splitting(X, y, n_split, split_fraction):
    ac_score = np.zeros(n_split)
    simple_score = np.zeros(n_split)
    categorical_features_indices = np.where(X.dtypes != np.float)[0]
    for i in range(n_split):
        X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size= split_fraction)
        train_pool = Pool(X_train, y_train, cat_features=categorical_features_indices)
        validate_pool = Pool(X_validation, y_validation, cat_features=categorical_features_indices)
        num_iter = 1000
        alpha = 0.2
        
        params = {
            'iterations': 500,
            'learning_rate': 0.1,
            'eval_metric': 'Accuracy',
    #'random_seed': 42,
            'logging_level': 'Silent',
            'use_best_model': False
        }

        model = CatBoostClassifier(**params)
        model.fit(train_pool, eval_set=validate_pool)

        best_model_params = params.copy()
        best_model_params.update({
            'use_best_model': True
        })
     

        ac_score[i] = accuracy_score(y_validation, model.predict(X_validation))
        
    return(ac_score)


def prepare_input(df4, cols):
    X = df4
    all_col = df4.columns
    for c in all_col:
        if (c in cols)==False:
            X = X.drop(c, axis=1)
            
    return X       
        

In [15]:
df4 = pd.read_csv('data4.csv')

X = prepare_input(df4, ['0'])

y = df4.Answer
n_split = 5
split_fraction = 0.75

multiple_splitting(X, y, n_split, split_fraction)

/var/folders/c5/n0m6fkj96kg58ft5bw7k66ym0000gw/T/ipykernel_23939/4280563854.py:4: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  categorical_features_indices = np.where(X.dtypes != np.float)[0]


array([0.76271186, 0.71186441, 0.77966102, 0.69491525, 0.69491525])

In [ ]:
print(y)